### Step 1: Load the model

Point model_id to model weight folder

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer,pipeline

In [ ]:
# get the model from the meta
model_id="./models_hf/7B-Chat-hf"

tokenizer = LlamaTokenizer.from_pretrained(model_id)

model =LlamaForCausalLM.from_pretrained(model_id, load_in_8bit=True, device_map='auto', torch_dtype=torch.float16)

### Step 2: Check base model

Run the base model on an example input:

In [ ]:
DEFAULT_SYSTEM_PROMPT_TEXT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


DEFAULT_SYSTEM_PROMPT = f"<s>[INST] <<SYS>>{DEFAULT_SYSTEM_PROMPT_TEXT}<</SYS>>"


In [ ]:
input_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-)
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

In [ ]:
prompt = f"{DEFAULT_SYSTEM_PROMPT}\n{input_prompt} [/INST]"

In [ ]:
model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

In [ ]:
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=100)
prompt = f"{DEFAULT_SYSTEM_PROMPT}\n{input_prompt} [/INST]"

result = pipe(prompt)
print(result[0]['generated_text'])